In [1]:
from dotenv import load_dotenv
from enum import Enum
from graphviz import Digraph
import instructor
from openai import OpenAI
import os
from pathlib import Path
from pprint import pprint as pp
from pydantic import BaseModel, Field, StringConstraints, conlist, field_validator
from typing import List, Optional, Union, Dict
from typing_extensions import Annotated

In [2]:
# load API key

dotenv_path = Path(r"C:\Storage\python_projects\ashvin\.env")
load_dotenv(dotenv_path=dotenv_path)
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

# main constants

GPT_MODEL = "gpt-4o" # points to latest GPT model

#instantiate client
client = instructor.from_openai(OpenAI(), mode=instructor.Mode.TOOLS)

In [3]:
# wrapper

def wrapper(system_prompt: str | None = None, user_context: str | list = None, response_model: BaseModel | None = None, max_retries: int = 3):
    """Wrapper function to generate LLM completion"""
    messages = []
    if system_prompt is not None:
        messages.append({"role": "system", "content": system_prompt})
    if user_context is not None:
        messages.append({"role": "user", "content": user_context})

    completion = client.chat.completions.create(
        model=GPT_MODEL,
        response_model=response_model,
        max_retries=max_retries,
        messages=messages
    )
    return completion

In [4]:
class Node(BaseModel):
    id: int
    label: str
    color: str

class Edge(BaseModel):
    source: int
    target: int
    label: str
    color: str = "black"

class KnowledgeGraph(BaseModel):
    nodes: List[Node] = Field(..., default_factory=list)
    edges: List[Edge] = Field(..., default_factory=list)

In [5]:
def visualize_knowledge_graph(kg: KnowledgeGraph):
    dot = Digraph(comment="Knowledge Graph")

    # Add nodes
    for node in kg.nodes:
        dot.node(str(node.id), node.label, color=node.color)

    # Add edges
    for edge in kg.edges:
        dot.edge(str(edge.source), str(edge.target), label=edge.label, color=edge.color)

    # Render the graph
    dot.render("knowledge_graph.gv", view=True)



In [38]:
system_prompt =""" 
Task: Based on the provided user context, create a comprehensive and intricate knowledge graph that is both broad and deep. 
Your graph must be as comprehensive as the best wikipedia articles.

Requirements:

Depth:

Include multiple layers of hierarchical relationships.
Represent the longest paths from root nodes to leaf nodes.

Breadth:

Ensure numerous direct connections and diverse relationships branching out from each node.
Exhibit high node and edge counts with a variety of node degrees.

Complexity:

Include detailed properties for nodes and edges, such as in-degree, out-degree, and centrality measures.
Ensure significant clustering where relevant.

Diversity:

Incorporate a wide range of domains and topics to ensure the graph is extensive and well-rounded.
Cover various aspects of knowledge comprehensively
.
Relevance:

Tailor the knowledge graph to accurately reflect and enhance the user's context.
Ensure the graph is both relevant and valuable to the user's needs.
"""

In [36]:
system_prompt = """
You are a world class knowledge graph algorithm capable of systematically extracting or inferring a knowledge graph from a given context.
You are expert at extracting or inferring detailed ontology classes, properties, individuals and axioms.
Generate a detailed knowledge graph that is broad and deep.

constructing and deconstructing structured and unstructured data from a context or your archives.



You are expert at translating and connecting this context into extremely detailed, precise and accurate nodes and edges.
You will produce long lists of nodes and edges at multiple levels
Before you call the function, think step-by-step to get a better understanding of the problem.
"""

system_prompt = """
You are a world class ontology algorithm capable of systematically extracting or inferring an ontology from a given context.
You are an expert at extracting or inferring detailed lists of ontology classes, properties, individuals and axioms. 
Each item and each list is precise, granular, comprehensive and complete.
Your purpose is to teach this ontology to a skilled individual without the domain knowledge.
Generate a dense, rich articulation with encyclopedic breadth and depth of an OWL ontology.
Always think broadly about a wide variety of aspects and factors then think deeply about each factor and aspect.
Before you call the function, think step-by-step to get a better understanding of the problem.
"""

In [47]:
user_context = "I want to learn all about quantum mechanics"

In [48]:
response = wrapper(system_prompt=system_prompt, user_context=user_context, response_model=KnowledgeGraph, max_retries=3)

In [49]:
visualize_knowledge_graph(response)